# T4-OPT: Train LLM with QLoRA

This notebook demonstrates QLoRA fine-tuning on a T4 GPU.

## Steps:
1. Load and prepare dataset
2. Configure QLoRA training
3. Train the model
4. Save checkpoint


In [ ]:
import sys
sys.path.append('/content/t4opt')

from training.qlora import QLoRATrainer, QLoRAConfig
from training.dataset import DatasetManager
from utils.memory import MemoryManager
from utils.config import Config

# Check memory
MemoryManager.print_memory_summary()


In [ ]:
# Configuration for T4
config = QLoRAConfig(
    model_name="microsoft/phi-2",  # or "google/gemma-2b-it"
    output_dir="./checkpoints/phi-2-qlora",
    max_seq_length=1024,
    micro_batch_size=1,
    gradient_accumulation_steps=16,
    num_epochs=3,
    learning_rate=2e-4,
    lora_r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    use_gradient_checkpointing=True,
    fp16=True
)

print("Training Configuration:")
for key, value in config.__dict__.items():
    print(f"  {key}: {value}")


In [ ]:
# Load dataset
dataset_manager = DatasetManager()
dataset_info = dataset_manager.load_dataset(
    dataset_name="alpaca",
    max_samples=1000  # Limit for T4
)

print(f"Dataset loaded: {dataset_info['num_samples']} samples")


In [ ]:
# Initialize trainer
trainer = QLoRATrainer(config=config)

# Load model
model, tokenizer = trainer.load_model()


In [ ]:
# Tokenize dataset
tokenized_dataset = dataset_manager.tokenize_dataset(
    dataset_info['dataset'],
    tokenizer,
    max_length=config.max_seq_length
)

print(f"Tokenized dataset: {len(tokenized_dataset)} samples")


In [ ]:
# Train model
print("Starting training...")
training_result = trainer.train(tokenized_dataset)

print("\nTraining Results:")
print(f"  Final Loss: {training_result['train_loss']:.4f}")
print(f"  Training Time: {training_result['train_runtime']:.2f} seconds")
print(f"  Samples/sec: {training_result['train_samples_per_second']:.2f}")
print(f"  Output Directory: {training_result['output_dir']}")


In [ ]:
# Check memory after training
MemoryManager.print_memory_summary()
